<div class="alert alert-success">
    Hi Cooper! I have approved your initial report for the final project. Great work! Just wanted to add some helpful pointers:
    <br/><br/>
If end date is empty, assume that the customer is still with the company (no churn)
    <br/><br/>
    When merging dataframes make sure that you do not miss any customers since some ID’s may not exist in certain tables. You can treat these missing cases as that customer not signing up for the specific service
<br/><br/>
    Your target variable should be Churn / No Churn, this will be a binary classification task
<br/><br/>
    Make sure that the validation set is used to tune hyperparameters, good idea to use gridsearch
<br/><br/>
    There does indeed exist a class imbalance in the data, make sure this is accounted for
<br/><br/>
    Careful with data leakage, make sure to not include features that may be collinear (i.e if using one-hot encoding for categorical variables, drop one of the columns) or reference the target variable in some way (see topic on data leakage)
<br/><br/>
    Good choice of models to try
<br/><br/>
    Good EDA plan
<br/><br/>
    The goal of this project is to understand why customers churned or stopped their service. And we can use machine learning to identify features of the data that had high predictive power when determining which customers will churn. And if we know that, what can we do as a company to prevent ongoing churn (this part is more of a discussion question for your end
<br/><br/>
-Yervand, Data Science Tutor
</div>

In [1]:
# import libraries
import pandas as pd

In [2]:
# Load the data
contract = pd.read_csv('/datasets/final_provider/contract.csv')
internet = pd.read_csv('/datasets/final_provider/internet.csv')
personal = pd.read_csv('/datasets/final_provider/personal.csv')
phone = pd.read_csv('/datasets/final_provider/phone.csv')

In [3]:
# Perform basic analysis for the contract set
display(contract)
print(contract.info())
len(contract[contract['EndDate'] == 'No']) / len(contract['EndDate'])

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65
...,...,...,...,...,...,...,...,...
7038,6840-RESVB,2018-02-01,No,One year,Yes,Mailed check,84.80,1990.5
7039,2234-XADUH,2014-02-01,No,One year,Yes,Credit card (automatic),103.20,7362.9
7040,4801-JZAZL,2019-03-01,No,Month-to-month,Yes,Electronic check,29.60,346.45
7041,8361-LTMKD,2019-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Mailed check,74.40,306.6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB
None


0.7346301292063041

Notes:
- No missing values
- ~73% of customers have not left yet, meaning the classes are unbalanced
- Need to change the column names to lowercase just for consistency

Tasks:
- Change the 'TotalCharges' column to float
- Change the 'BeginDate' to datetime

Feature engineering:

- Need to perform one-hot encoding on columns: 'Type', 'PaperlessBilling', and 'PaymentMethod'
- Make separate column based on 'EndDate' for whether or not the customer has left
- Perform class balancing

In [4]:
# Perform basic analysis for the internet set
display(internet)
print(internet.info())

,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...
5512,6840-RESVB,DSL,Yes,No,Yes,Yes,Yes,Yes
5513,2234-XADUH,Fiber optic,No,Yes,Yes,No,Yes,Yes
5514,4801-JZAZL,DSL,Yes,No,No,No,No,No
5515,8361-LTMKD,Fiber optic,No,No,No,No,No,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB
None


Notes:
- Uses same form of ID as the contract set, easily combined
- Not every customer utilizes their internet service, will have to fill in na rows for those that don't when I combine the sets
- Just need to use one-hot encoding for every column except customerId
- Change column names to lowercase

In [5]:
# Perform basic analysis for the personal set
display(personal)
print(personal.info())

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No
...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes
7039,2234-XADUH,Female,0,Yes,Yes
7040,4801-JZAZL,Female,0,Yes,Yes
7041,8361-LTMKD,Male,1,Yes,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB
None


Notes:
- Set uses the same form of id and the same number of users as the contract set, easily combined
- Need to feature engineer 'gender', 'Partner', and 'Dependents' columns with one-hot encoding
- Change columns to lowercase

In [6]:
# Perform basic analysis for the phone set
display(phone)
print(phone.info())

,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes
...,...,...
6356,2569-WGERO,No
6357,6840-RESVB,Yes
6358,2234-XADUH,Yes
6359,8361-LTMKD,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB
None


Notes:
- Uses the same form of ID
- Not every user with a contract utilizes their phone service, will have to fillna some rows
- Feature engineer the 'MultipleLines' column with one-hot encoding, including a column to show customers without the phone service

# Proposed Work Plan:

Goal:
Develop a predictive model to help Interconnect forecast client churn. By analyzing customer data, including plan and contract details, I aim to identify patterns indicating when clients are likely to leave. This will allow the marketing team to offer targeted promotions to retain at-risk customers, improving retention and reducing churn.

Steps:

1. Download the 4 datasets
2. Perform basic analysis on each set to understand their structure and contents
3. Preprocess the data:
    - Standardize column names for consistency
    - Convert columns to their appropriate data types
    - Merge the datasets into one comprehensive dataset
4. Conduct Exploratory Data Analysis (EDA) to gain deeper insights into the data
5. Engineer features:
    - Balance the target class (churned vs. not churned)
    - Apply one-hot encoding for categorical variables
6. Divide the dataset into training and testing sets to evaluate model performance
7. Select and Train Models:
    - Choose appropriate algorithms for churn prediction (e.g., Logistic Regression, Decision Trees, Random Forest, Gradient Boosting, etc.)
    - Train the models using the training dataset
8. Evaluate and optimize model performance: 
    - Use metrics such as AUC-ROC and accuracy
    - Compare different models to determine which performs best
    - Tune hyperparameters for optimal results
9. Validate the model:
    - Test the model on the testing dataset to confirm its reliability
    - Perform cross-validation for further evaluation
10. Interpret results:
    - Analyze feature importance to understand which factors contribute most to churn predictions